In [51]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from langchain_groq import ChatGroq
import os
import asyncio
load_dotenv()
grok_api_key = os.getenv("GROQ_API_KEY")
ALPHA_VANTAGE_API_KEY =os.getenv("ALPHA_VANTAGE_API_KEY")
alpha_client = MultiServerMCPClient(
    {
    "alphavantage": {
      "command": "docker",
      "args": [
        "run",
        "-i",
        "-e",
        "ALPHA_VANTAGE_API_KEY",
        "mcp/alpha-vantage"
      ],
      "env": {
        "ALPHA_VANTAGE_API_KEY": "{ALPHA_VANTAGE_API_KEY}"
      },
      "transport":"stdio"
    }
    }
)
yahoo_client = MultiServerMCPClient(
    {
    "yfinance": {
      "command": "uv",
      "args": [
        "--directory",
        "yahoo-finance-mcp",
        "run",
        "server.py"
      ],
       "transport":"stdio"
    }
  }
)
alpha_client0=MultiServerMCPClient(
    {
    "alphavantage": {
      "command": "uv",
      "args": [
        "--directory",
        "alphavantage",
        "run",
        "alphavantage"
      ],
      "env": {
        "ALPHAVANTAGE_API_KEY": "{ALPHA_VANTAGE_API_KEY}"
      },
      "transport":"stdio"
    }
  }
)
edgar_client = MultiServerMCPClient(
    {
    "mcp-edgar-sec": {
      "command": "uv",
      "args": [
        "run",
        "--with",
        "mcp",
        "--with",
        "edgartools",
        "--with",
        "pandas",
        "--with",
        "requests",
        "--with",
        "nest_asyncio",
        "--directory",
        "mcp-edgar-sec",
        "mcp",
        "run",
        "src/server.py"
      ],
      "transport":"stdio",
    }
  }
)
async def main():
    tools= await edgar_client.get_tools()
    llm=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",api_key=grok_api_key)
    agent=create_react_agent(model=llm,tools=tools)
    response =await agent.ainvoke({"messages": [HumanMessage(content="""get filings content of microsoft"""),]},{"recursion_limit": 25})
    return response


In [ ]:
response = await main()

In [50]:
resp=response['messages']
for i in range(len(resp)):
    if isinstance(resp[i], AIMessage):
        print(f"AI: {resp[i].content}")
    elif isinstance(resp[i], HumanMessage):
        print(f"Human: {resp[i].content}")
    else:
        print(f"TOOL: {resp[i].content}")

Human: get filings of microsoft
AI: 
TOOL: {"filings": [{"form": "4", "filing_date": "2025-05-22", "accession_number": "0001062993-25-010134", "isXBRL": false}, {"form": "144", "filing_date": "2025-05-21", "accession_number": "0002007317-25-000429", "isXBRL": false}, {"form": "4", "filing_date": "2025-05-20", "accession_number": "0001062993-25-009901", "isXBRL": false}, {"form": "144", "filing_date": "2025-05-19", "accession_number": "0001959173-25-003326", "isXBRL": false}, {"form": "4", "filing_date": "2025-05-16", "accession_number": "0001062993-25-009723", "isXBRL": false}, {"form": "144", "filing_date": "2025-05-15", "accession_number": "0001959173-25-003240", "isXBRL": false}, {"form": "144", "filing_date": "2025-05-15", "accession_number": "0001959173-25-003239", "isXBRL": false}, {"form": "4", "filing_date": "2025-05-06", "accession_number": "0001062993-25-008701", "isXBRL": false}, {"form": "144", "filing_date": "2025-05-05", "accession_number": "0001950047-25-002724", "isXBRL

In [ ]:
type(resp)

langchain_core.messages.tool.ToolMessage